## 보팅방식 모델 구현 <hr>
- 데이터셋 : sklearn.datasets의 breast_cancer
- 유 형 : 지도학습 + 분류
- 방 법 : LogisticRegression, DecisionTreeClassifier, SVC
- 학습 데이터셋 : 동일한 데이터셋으로 3개의 모델 학습 진행

[1] 모듈 로딩 및 데이터 준비 <hr>

In [5]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer

In [6]:
X, y = load_breast_cancer(as_frame = True, return_X_y = True)

In [7]:
X.shape, y.shape

((569, 30), (569,))

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state = 7,
                                                    test_size = 0.2,
                                                    stratify = y)

[2] 학습 진행 <hr>

[2-1] 앙상블 보팅 학습에 사용할 모델 인스턴스 생성

In [9]:
# LogisticRegression 인스턴스
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver = 'liblinear')
lr_model.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [18]:
# DecisionTree 인스턴스
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state = 7)
dt_model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=7)

In [11]:
# SVC 인스턴스
from sklearn.svm import SVC

svc_model = SVC()
svc_model.fit(X_train, y_train)

SVC()

[2-2] 

In [19]:
from sklearn.ensemble import VotingClassifier

# 동일 데이터셋을 병렬 학습 진행할 모델 리스트 선정 및 결과 결정 방법 설정
vt_models = VotingClassifier(estimators=[('lr_model', lr_model),
                                        ('dt_model', dt_model),
                                        ('svc_model', svc_model)],
                                        verbose = True)

In [20]:
# 동일 데이터셋을 전달해서 3개의 모델 동시에 학습 진행
vt_models.fit(X_train, y_train)

[Voting] ................. (1 of 3) Processing lr_model, total=   0.0s
[Voting] ................. (2 of 3) Processing dt_model, total=   0.0s
[Voting] ................ (3 of 3) Processing svc_model, total=   0.0s


VotingClassifier(estimators=[('lr_model',
                              LogisticRegression(solver='liblinear')),
                             ('dt_model',
                              DecisionTreeClassifier(random_state=7)),
                             ('svc_model', SVC())],
                 verbose=True)

In [26]:
# 예측하기
new_data = pd.DataFrame([X_test.iloc[0]], columns = X_test.columns)  # 경고 메세지 피하기 위해서 열이름 지정한다.

# 다음 두개의 결과는 같다.
vt_models.predict(new_data)
vt_models.predict(X_test.iloc[0].to_frame().T)

array([0])

In [27]:
# 예측하기
vt_models.predict_proba(new_data)  # SVC probability 매개변수 True로 설정, 보팅 방식 soft로 변경하면 해결 => 아래 참고

AttributeError: predict_proba is not available when voting='hard'

In [22]:
# 보팅 인스턴스 내의 학습기들 => 접근 방법 (1)
vt_models.estimators_

[LogisticRegression(solver='liblinear'),
 DecisionTreeClassifier(random_state=7),
 SVC()]

In [29]:
# 보팅 인스턴스 내의 학습기들 => 접근 방법 (2)
vt_models.named_estimators_

{'lr_model': LogisticRegression(solver='liblinear'),
 'dt_model': DecisionTreeClassifier(random_state=7),
 'svc_model': SVC()}

In [33]:
for key, value in vt_models.named_estimators_.items():
    print(key, value.predict(new_data))

lr_model [0]
dt_model [0]
svc_model [0]


In [23]:
vt_models.estimators_[0].predict(X_test)

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1], dtype=int64)

- predict_proba 메서드로 확률 반환

In [41]:
# SVC 인스턴스
from sklearn.svm import SVC

svc_model = SVC(probability = True)
svc_model.fit(X_train, y_train)

SVC(probability=True)

In [42]:
# 동일 데이터셋으로 병렬 학습 진행할 모델 리스트 선정 및 결과 결정 방법 설정
vt_models = VotingClassifier(estimators=[('lr_model', lr_model),
                                        ('dt_model', dt_model),
                                        ('svc_model', svc_model)],
                                        verbose = True,
                                        voting = 'soft')

In [44]:
# 동일 데이터셋을 전달해서 3개의 모델 동시에 학습 진행
vt_models.fit(X_train, y_train)

[Voting] ................. (1 of 3) Processing lr_model, total=   0.0s
[Voting] ................. (2 of 3) Processing dt_model, total=   0.0s
[Voting] ................ (3 of 3) Processing svc_model, total=   0.0s


VotingClassifier(estimators=[('lr_model',
                              LogisticRegression(solver='liblinear')),
                             ('dt_model',
                              DecisionTreeClassifier(random_state=7)),
                             ('svc_model', SVC(probability=True))],
                 verbose=True, voting='soft')

In [45]:
# 예측하기
vt_models.predict_proba(new_data)  # SVC probability 매개변수 True로 설정, 보팅 방식 soft로 변경하면 해결

array([[9.99042369e-01, 9.57631108e-04]])